In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import numpy.random as rnd
import numpy.linalg as la
import polars as pl
import pandas as pd
import datetime as dt
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import networkx as nx
import raphtory as rp
import umap
import community
import torch
import torch_geometric as tg
from torch_geometric.data import Data
from torch_geometric.utils.convert import from_networkx
from torch_geometric.transforms import LargestConnectedComponents
from torch_geometric.utils import to_networkx, from_networkx
from torch_geometric.nn import Node2Vec, GCNConv, VGAE
import torch.nn.functional as F

In [3]:
from datasets import DataLoader

In [4]:
PATH = "./datasets/data/nas/"

# <font color="grey"> $\quad$ New Autonomous Systems dataset </font>

$\newcommand{\vct}[1]{\mathbf{#1}}$
$\newcommand{\mtx}[1]{\mathbf{#1}}$
$\newcommand{\e}{\varepsilon}$
$\newcommand{\norm}[1]{\|#1\|}$
$\newcommand{\minimize}{\mathrm{minimize}\quad}$
$\newcommand{\maximize}{\mathrm{maximize}\quad}$
$\newcommand{\subjto}{\quad\text{subject to}\quad}$
$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\N}{\mathbb{N}}$
$\newcommand{\Z}{\mathbb{Z}}$
$\newcommand{Prob}{\mathbb{P}}$
$\newcommand{Expect}{\mathbb{E}}$
$\newcommand{Cov}{\mathrm{Cov}}$
$\newcommand{Var}{\mathrm{Var}}$
$\newcommand{\trans}{T}$
$\newcommand{\ip}[2]{\langle {#1}, {#2} \rangle}$
$\newcommand{\zerovct}{\vct{0}}$
$\newcommand{\diff}[1]{\mathrm{d}{#1}}$
$\newcommand{\conv}{\operatorname{conv}}$
$\newcommand{\inter}{{\operatorname{int}}}$

### <font color="grey">  Table of Contents</font>

1. #### <a href='#chapter1'>Data</a>
2. #### <a href='#chapter2'>Embedding</a>
3. #### <a href='#chapter3'>Visualisation</a>

###  <a id='chapter1'> <font color="grey">1. Data </font></a>

The data can be accessed via the dataloader. It is saved in the datasets/data/nas directory in two parquet files. 

In [23]:
dl = DataLoader(source='nAS')

The data is stored in one dataframe for the nodes (including all the features) and one for the edges (including edge weights).

In [24]:
# Get the nodes
node_df = dl.get_nodes()
node_df.head()

timestamp,nodes,nodetype,country,asname
datetime[μs],str,str,str,str
2024-09-14 00:00:00,"""AS7029""","""asn""","""US""","""WINDSTREAM"""
2024-09-14 00:00:00,"""AS32984""","""asn""","""US""","""RUELALA-INC"""
2024-09-14 00:00:00,"""AS136106""","""asn""","""ID""","""FIBERSTAR-AS-I"""
2024-09-14 00:00:00,"""AS58495""","""asn""","""ID""","""HSPNET-AS-I"""
2024-09-14 00:00:00,"""AS3491""","""asn""","""US""","""BTN-ASN"""


In [7]:
edge_df = dl.get_edges()
edge_df.head()

timestamp,source,dest,weight
datetime[μs],str,str,i64
2024-09-14 00:00:00,"""AS7029""","""AS32984""",1
2024-09-14 00:00:00,"""AS7029""","""AS19692""",1
2024-09-14 00:00:00,"""AS7029""","""AS55037""",1
2024-09-14 00:00:00,"""AS7029""","""AS1820""",1
2024-09-14 00:00:00,"""AS7029""","""AS16265""",1


Ultimately, working with the people at Pometry, we want to use the Raphtory graph format

In [8]:
# Raphtory format
g = dl.get_graph()

In [9]:
print("Stats on the graph structure:")

number_of_nodes = g.count_nodes()
number_of_edges = g.count_edges()
total_interactions = g.count_temporal_edges()

print("Number of nodes (AS nodes):", number_of_nodes)
print("Number of unique edges (src,dst):", number_of_edges)
print("Total interactions (edge updates):", total_interactions)

print("Stats on the graphs time range:")

earliest_datetime = g.earliest_date_time
latest_datetime = g.latest_date_time

print("Earliest datetime:", earliest_datetime)
print("Latest datetime:", latest_datetime)

Stats on the graph structure:
Number of nodes (AS nodes): 85428
Number of unique edges (src,dst): 914346
Total interactions (edge updates): 16717484
Stats on the graphs time range:
Earliest datetime: 2024-09-14 00:00:00+00:00
Latest datetime: 2024-10-13 00:00:00+00:00


In [10]:
print("The node features are: ", g.nodes.properties.keys())

The node features are:  ['asname', 'country', 'nodetype']


We need to prepare the data for the use with pytorch-geometric.

In [25]:
dates = dl.get_dates()

In [26]:
dates

[datetime.datetime(2024, 9, 14, 0, 0),
 datetime.datetime(2024, 9, 15, 0, 0),
 datetime.datetime(2024, 9, 16, 0, 0),
 datetime.datetime(2024, 9, 17, 0, 0),
 datetime.datetime(2024, 9, 18, 0, 0),
 datetime.datetime(2024, 9, 19, 0, 0),
 datetime.datetime(2024, 9, 20, 0, 0),
 datetime.datetime(2024, 9, 21, 0, 0),
 datetime.datetime(2024, 9, 22, 0, 0),
 datetime.datetime(2024, 9, 23, 0, 0),
 datetime.datetime(2024, 9, 24, 0, 0),
 datetime.datetime(2024, 9, 25, 0, 0),
 datetime.datetime(2024, 9, 26, 0, 0),
 datetime.datetime(2024, 9, 27, 0, 0),
 datetime.datetime(2024, 9, 28, 0, 0),
 datetime.datetime(2024, 9, 29, 0, 0),
 datetime.datetime(2024, 9, 30, 0, 0),
 datetime.datetime(2024, 10, 1, 0, 0),
 datetime.datetime(2024, 10, 2, 0, 0),
 datetime.datetime(2024, 10, 3, 0, 0),
 datetime.datetime(2024, 10, 4, 0, 0),
 datetime.datetime(2024, 10, 5, 0, 0),
 datetime.datetime(2024, 10, 6, 0, 0),
 datetime.datetime(2024, 10, 7, 0, 0),
 datetime.datetime(2024, 10, 8, 0, 0),
 datetime.datetime(2024, 

In [28]:
# Encode nodes
nodes = dl.get_node_list()
node_dict = dict(zip(nodes,range(len(nodes))))

In [29]:
# Encode country codes
cc = pl.read_csv(PATH+'country_codes.csv')
ccdict = dict(zip(cc["alpha-2"].to_list(), cc["country-code"].cast(pl.Int64).to_list()))
features = dl.nodes.select(
    pl.col("country").replace(old=pl.Series(ccdict.keys()), new=pl.Series(ccdict.values())).cast(pl.Int64)
).to_numpy()
features

array([[840],
       [840],
       [360],
       ...,
       [152],
       [156],
       [ 76]])

In [30]:
# Create pytorch-geometric Data object
tg_graphs = {}
for d in tqdm(dates):
    edges = dl.edges.filter(pl.col('timestamp')==d).select(
        pl.col('source').replace(old=pl.Series(node_dict.keys()), new=pl.Series(node_dict.values())).cast(pl.Int64),
        pl.col('dest').replace(old=pl.Series(node_dict.keys()), new=pl.Series(node_dict.values())).cast(pl.Int64)
    ).to_numpy()
    edge_index = torch.tensor([tuple(x) for x in edges], dtype=torch.long).t().contiguous()
    tgraph = Data(edge_index=edge_index)
    # Add features - problem is that for the embedding we only want those present at a given time
    tgraph.x = torch.from_numpy(features).float()
    tg_graphs[d] = tgraph

  0%|          | 0/30 [00:00<?, ?it/s]

A description of how pytorch-geometric deals with its graph data structure can be found [here](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html).

In [16]:
features.shape

(2538974, 1)

In [32]:
data = tg_graphs[dates[0]]

###  <a id='chapter2'> <font color="grey">2. Embedding </font></a>

For the embedding, we use the architecture of a Variational Graph Autoencoder. Given a graph $G=(V,E)$ with $|V|=n$ nodes and node features $\vct{x}_i\in \R^d$, $i\in [n]$, denote by $\vct{X}=[\vct{x}_1,\dots,\vct{x}_n]^T\in \R^{n\times d}$ the features matrix and by $A=(a_{ij})\in \{0,1\}^{n\times n}$ the adjacency matrix of the graph. The **encoder** produces latent representations $\vct{z}_i\in \R^k$ for $i\in [n]$, which are sampled from the inference model
\begin{equation*}
  q(\vct{z}_i \ | \ \vct{X},\vct{A}) = \mathcal{N}(\vct{z}_i \ | \ \vct{\mu}_i,\mathrm{diag}(\vct{\sigma}_i)).
\end{equation*}
The means $\mu_i$ and variances $\mathrm{diag}(\vct{\sigma}_i)$ are parametrized using an encoder network, for example, a graph convolutional neural network (GCN). Denoting by $\vct{Z}=[\vct{z}_1,\dots,\vct{z}_n]^T$ the matrix of latent represenations and by $\vct{\mu}$ and $\vct{\sigma}$ the matrices representing the means and variances, we have
\begin{equation*}
  \vct{\mu} = \mathrm{GCN}_{\mu}(\vct{X},\vct{A}), \quad \quad \log \vct{\sigma} = \mathrm{GCN}_{\sigma}(\vct{X},\vct{A}).
\end{equation*}
The **generative model** is a distribution on the adjacency matrix,
\begin{equation*}
  p(\mtx{A}\ | \ \vct{Z}) = \prod_{i,j} p(a_{ij} \ | \ \vct{z}_i,\vct{z}_j).
\end{equation*}
It is convenient to use
\begin{equation*}
  p(a_{ij}=1 \ | \ \vct{z}_i,\vct{z}_j) = \sigma(\vct{z}_i^T\vct{z}_j),
\end{equation*}
where $\sigma$ is the logistic sigmoid. In order to train the model, we optimize the evidence lower bound
\begin{equation*}
  \mathcal{L} = \Expect_{q(\vct{Z}\ | \ \vct{X},\vct{A})}[\log p(\mtx{A}\ | \ \mtx{Z})]-\mathrm{D}_{\mathrm{KL}}(q(\mtx{Z}\ | \ \mtx{X},\mtx{A}) \ \| \ p(\mtx{Z})).
\end{equation*}

In [33]:
class Encoder(torch.nn.Module):
    """
    Implement a Graph Convolutional Network (GCN) as encoder
    """
    def __init__(self, dim, num_node_features, hidden_dim=32, cached=True, bias=True, add_self_loops=True, normalize=True):
        super().__init__()
        self.x_conv = tg.nn.GCNConv(num_node_features, 
                                    hidden_dim, 
                                    cached=cached, 
                                    bias=bias, 
                                    add_self_loops=add_self_loops,
                                    normalize=normalize)
        self.mean_conv = tg.nn.GCNConv(hidden_dim, 
                                       dim, 
                                       cached=cached,
                                       bias=bias, 
                                       add_self_loops=add_self_loops,
                                       normalize=normalize)
        self.var_conv = tg.nn.GCNConv(hidden_dim, 
                                      dim, 
                                      cached=cached, 
                                      bias=bias, add_self_loops=add_self_loops,
                                      normalize=normalize)

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index

        x = self.x_conv(x, edge_index)
        x = F.relu(x)

        mu = self.mean_conv(x, edge_index)
        sigma = self.var_conv(x, edge_index)
        return mu, sigma

In [34]:
model = VGAE(encoder=Encoder(10, data.num_node_features))
#model = VGAE(encoder=VGAEconv(2, test_data.num_node_features))

In [35]:
def VGAE_loss(model, data):
    return model.recon_loss(model.encode(data), data.edge_index) + model.kl_loss() / data.num_nodes

In [36]:
def train(data, model, loss_fun, num_epochs=100, verbose=True, lr=0.01, logger=lambda loss: None):
    losses = []
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    # schedule = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs)
    for e in tqdm(range(num_epochs)):
        optimizer.zero_grad()
        loss = loss_fun(model, data)
        loss.backward()
        optimizer.step()
        losses.append(float(loss))
        if verbose:
            if not e % 20:
                print(f'epoch {e}: loss={loss.item()}')
        # schedule.step()
    return model, losses

In [40]:
model, losses = train(data, model, VGAE_loss, num_epochs=100)

  0%|          | 0/100 [00:00<?, ?it/s]

epoch 0: loss=33.53376770019531
epoch 20: loss=33.47657012939453
epoch 40: loss=33.478885650634766
epoch 60: loss=33.48881530761719
epoch 80: loss=33.47344207763672


TypeError: cannot unpack non-iterable VGAE object

In [ ]:
plt.plot(losses)
plt.show()

In [ ]:
embedding = model.encode(data).detach().numpy()
embedding.shape

In [ ]:
def VGAE_patch_embeddings(patch_data, dim=2, hidden_dim=32, num_epochs=100, decoder=None, device='cpu', lr=0.01):
    patch_list = []
    models = []
    for patch in patch_data:
        if patch.x is None:
            patch.x = speye(patch.num_nodes)
            print(f"training patch with {patch.edge_index.shape[1]} edges")   #added [i] to every patch
            model = tg.nn.VGAE(encoder=VGAEconv(dim, patch.x.shape[1], hidden_dim=hidden_dim), decoder=decoder).to(device)
            patch.to(device)

        model = train(patch, model, VGAE_loss, num_epochs=num_epochs, lr=lr)
        with torch.no_grad():
            model.eval()
            coordinates = model.encode(patch).to('cpu').numpy()
            models.append(model)
            patch_list.append(l2g.Patch(patch.nodes.to('cpu').numpy(), coordinates))
    return patch_list, models

###  <a id='chapter3'> <font color="grey">3. Visualisation </font></a>

In [ ]:
# Use UMAP to visualise the graph embeddings for different days
reducer = umap.UMAP()

In [ ]:
embedding